In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import healpy as hp
import os
from tqdm import tqdm

from msfm import fiducial_pipeline
from msfm.utils import peak_statistics, files, maps

23-12-20 12:43:57   imports.py INF   Setting up healpy to run on 8 CPUs 
23-12-20 12:43:58   imports.py INF   Setting up healpy to run on 8 CPUs 
23-12-20 12:43:58   imports.py INF   Setting up healpy to run on 8 CPUs 


In [3]:
repo_dir = "/cluster/home/athomsen/dlss/repos/multiprobe-simulation-forward-model/"
conf = "/cluster/home/athomsen/dlss/repos/multiprobe-simulation-forward-model/configs/v6/linear_bias.yaml"
conf = files.load_config(conf)

In [4]:
n_bins = conf["analysis"]["peak_statistics"]["n_bins"]
smoothing_scales = conf["analysis"]["peak_statistics"]["smoothing_scales"]

def data_vector_to_peaks(data_vector, patch_pix):
    full_sky = np.full((conf["analysis"]["n_pix"], data_vector.shape[-1]), hp.UNSEEN)
    full_sky[patch_pix] = data_vector
    full_sky = maps.tomographic_reorder(full_sky, n2r=True)
    
    peaks = peak_statistics.get_peaks(
        full_sky,
        binning_dir=os.path.join(repo_dir, conf["dirs"]["peak_binning"]),
        n_side=conf["analysis"]["n_side"],
        n_bins=conf["analysis"]["peak_statistics"]["n_bins"],
        smoothing_scales=conf["analysis"]["peak_statistics"]["smoothing_scales"],
        with_cross=True,
        save_binning=True,
    )

    return peaks

In [5]:
# tfr_pattern = "/cluster/work/refregier/athomsen/CosmoGrid/DESY3/v6/linear_bias/tfrecords/fiducial/DESy3_fiducial_000.tfrecord"
tfr_pattern = "/cluster/work/refregier/athomsen/CosmoGrid/DESY3/v4/large_scales/tfrecords/DESy3_fiducial_000.tfrecord"

pipe = fiducial_pipeline.FiducialPipeline(
    conf,
    params=[],
    with_lensing=True,
    with_clustering=True,
    with_padding=False,
    apply_norm=False,
    apply_m_bias=True,
    shape_noise_scale=1.0,
    poisson_noise_scale=1.0,
)
dset = pipe.get_dset(
    tfr_pattern=tfr_pattern,
    local_batch_size=1,
    n_noise=1,
    n_readers=1,
    n_prefetch=0,
    is_eval=True,
)

23-12-20 12:43:59     files.py INF   Loaded the pixel file /cluster/home/athomsen/dlss/repos/multiprobe-simulation-forward-model/data/DESY3_pixels_512.h5 
23-12-20 12:43:59     files.py INF   Loaded the pixel file /cluster/home/athomsen/dlss/repos/multiprobe-simulation-forward-model/data/DESY3_pixels_512.h5 
23-12-20 12:44:00 fiducial_pip WAR   Evaluation mode is activated: the random seed is fixed, the shuffle arguments ignored, and the dataset is not repeated 
23-12-20 12:44:00 fiducial_pip INF   Interleaving with n_readers = 1 


2023-12-20 12:44:00.010392: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
/cluster/apps/nss/gcc-8.2.0/python/3.10.4/x86_64/lib64/python3.10/site-packages/tensorflow/python/data/ops/dataset_ops.py:2210: UserWarning: The `deterministic` argument has no effect unless the `num_parallel_calls` argument is specified.
  warnings.warn("The `deterministic` argument has no effect unless the "


23-12-20 12:44:00 fiducial_pip INF   Batching into 1 elements locally 
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'NoneType' object has no attribute '_fields'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'NoneType' object has no attribute '_fields'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'NoneType' object has no attribute '_fields'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
23-12-20 12:44:02 fiducial_pip WAR   Tracing _augmentations 
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
23-12-20 12:44:04 fiducial_pip INF   Running on the data_vectors.keys() = dict_keys(['kg_fiducial', 'dg_fiducial', 'i_example', 'sn', 'pn', 'i_noise']) 
23-12-20 12:44:04 fiducial_pip WAR   Tracing _lensing_augmentations 
23-12-20 12:44:04 fiducial_pip WAR   Tracing _clustering_augmentations 
23-12-20 12:44:04 fiducial_pip INF   Removing the padding 
23-12-20 12:44:04 fiducial_pip INF   Successfully generated the fiducial training set with element_spec (TensorSpec(shape=(None, None, 8), dtype=tf.float32, name=None), (TensorSpec(shape=(None,), dtype=tf.int64, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None)

In [6]:
for data_vector, (i_example, i_noise) in tqdm(dset.take(1), desc="Loop over examples"):
    # only consider a single example
    data_vector = np.squeeze(data_vector.numpy())
    
    data_vector_to_peaks(data_vector, pipe.patch_pix)


Loop over examples: 0it [00:00, ?it/s]

2023-12-20 12:44:07,729 -       map.py: 799 - WARNING - Weights not set for instance 0. Using equal weighting.


2023-12-20 12:44:07,737 -       map.py: 799 - WARNING - Weights not set for instance 1. Using equal weighting.


23-12-20 12:44:17 peak_statist INF   Saved binning scheme for 0x0 to /cluster/home/athomsen/dlss/repos/multiprobe-simulation-forward-model/data/peaks/binning/bin_centers_0x0.npy and /cluster/home/athomsen/dlss/repos/multiprobe-simulation-forward-model/data/peaks/binning/bin_edges_0x0.npy 
2023-12-20 12:44:17,708 -   summary.py: 985 -    INFO - Down sampled data vectors for statistic CrossPeaks from 1000 bins to 15 bins


INFO:estats.summary:Down sampled data vectors for statistic CrossPeaks from 1000 bins to 15 bins
ic| bin_edges: array([[-0.00495137, -0.00307385, -0.00119634,  0.00068117,  0.00255868,
                        0.0044362 ,  0.00631371,  0.00819122,  0.01006874,  0.01194625,
                        0.01382376,  0.01570127,  0.01757879,  0.0194563 ,  0.02133381,
                        0.02321133]])


...

INFO:estats.summary:Down sampled data vectors for statistic CrossPeaks from 1000 bins to 15 bins


2023-12-20 12:49:45,639 -       map.py: 799 - WARNING - Weights not set for instance 0. Using equal weighting.


2023-12-20 12:49:45,642 -       map.py: 799 - WARNING - Weights not set for instance 1. Using equal weighting.


23-12-20 12:49:55 peak_statist INF   Saved binning scheme for 7x7 to /cluster/home/athomsen/dlss/repos/multiprobe-simulation-forward-model/data/peaks/binning/bin_centers_7x7.npy and /cluster/home/athomsen/dlss/repos/multiprobe-simulation-forward-model/data/peaks/binning/bin_edges_7x7.npy 
2023-12-20 12:49:55,147 -   summary.py: 985 -    INFO - Down sampled data vectors for statistic CrossPeaks from 1000 bins to 15 bins


INFO:estats.summary:Down sampled data vectors for statistic CrossPeaks from 1000 bins to 15 bins
Loop over examples: 1it [05:50, 350.33s/it]
